# CLI

> The Plash CLI tool

This module implements the Plash CLI functions.
As they're built with fastcore `@call_parse` they can also be used in python directly.

In [ ]:
#| default_exp cli

In [ ]:
#| export
from fastcore.all import *
from fastcore.xdg import *
import secrets, webbrowser, json, httpx, io, tarfile, random, string
from pathlib import Path
from uuid import uuid4
from time import time, sleep
import io, os, re, tarfile, tomllib

from plash_cli import __version__

In [ ]:
#| hide
from tempfile import TemporaryDirectory
from nbdev.showdoc import show_doc

## Helpers -

In [ ]:
#| export
PLASH_CONFIG_HOME = xdg_config_home() / 'plash_config.json'
PLASH_DOMAIN = os.getenv("PLASH_DOMAIN","pla.sh")  # pla.sh plash-dev.answer.ai localhost:5002
IN_DEV = PLASH_DOMAIN.endswith('.internal')

In [ ]:
#| export
def _get_client(cookie_file):
    client = httpx.Client(verify=not IN_DEV)
    if not cookie_file.exists():
        raise FileNotFoundError("Plash config not found. Please run plash_login and try again.")
    cookies = Path(cookie_file).read_json()
    client.cookies.update(cookies)
    client.headers.update({'X-PLASH': 'true', 'User-Agent': f'plash_cli/{__version__}'})
    return client

In [ ]:
#| export
def _mk_auth_req(url:str, method:str='get', timeout=300., **kwargs):
    r = getattr(_get_client(PLASH_CONFIG_HOME), method)(url, timeout=timeout, **kwargs)
    if r.status_code == 200: return r
    else: print(f'Failure: {r.headers["X-Plash-Error"]}')

In [ ]:
#| export
def _get_app_name(path:Path):
    plash_app = Path(path) / '.plash'
    if not plash_app.exists(): raise FileNotFoundError(f"File not found: {plash_app=}")
    env = parse_env(fn=plash_app)
    if name:=env.get("PLASH_APP_NAME"): return name
    if aid :=env.get('PLASH_APP_ID'): 
            plash_app.write_text(f"export PLASH_APP_NAME={aid}")
            return aid
    raise RuntimeError(f"{plash_app=} did not have a PLASH_APP_NAME")


In [ ]:
#| export
def _endpoint(sub='', rt=''):
    p = "http" if "localhost" in PLASH_DOMAIN else "https"
    return f"{p}://{sub}{'.' if sub else ''}{PLASH_DOMAIN}{rt}"

In [ ]:
#| export
def _is_included(path):
    "Returns True if path should be included in deployment"
    if path.name.startswith('.'): return False
    if path.suffix == '.pyc': return False
    excludes = {'.git', '__pycache__', '.gitignore', '.env', 
                '.pytest_cache', '.venv', 'venv', '.ipynb_checkpoints',
                '.vscode', '.idea', '.sesskey'}
    return not any(p in excludes for p in path.parts)

In [ ]:
#| export
#| hide
class PlashError(Exception): pass

In [ ]:
#| export
def _poll_cookies(paircode, interval=1, timeout=180):
    "Poll server for token until received or timeout"
    start = time()
    client = httpx.Client(verify=not IN_DEV)
    url = _endpoint(rt=f"/cli_token?paircode={paircode}")
    while time()-start < timeout:
        resp = client.get(url).raise_for_status()
        if resp.text.strip(): return dict(client.cookies)
        sleep(interval)
     

In [ ]:
#| export   
@call_parse
def login():
    "Authenticate CLI with server and save config"
    paircode = secrets.token_urlsafe(16)
    if not IN_DEV:
        login_url = httpx.get(_endpoint(rt=f"/cli_login?paircode={paircode}")).text
        print(f"Opening browser for authentication:\n{login_url}\n")
        webbrowser.open(login_url)
    
    cookies = _poll_cookies(paircode)
    if cookies:
        Path(PLASH_CONFIG_HOME).write_text(json.dumps(cookies))
        print(f"Authentication successful! Config saved to {PLASH_CONFIG_HOME}")
    else: print("Authentication timed out.")

CLI usage:

In [ ]:
%%bash
plash_login --help

usage: plash_login [-h]

Authenticate CLI with server and save config

options:
  -h, --help  show this help message and exit


In [ ]:
#| export
pat = r'(?m)^# /// (?P<type>[a-zA-Z0-9-]+)$\s(?P<content>(^#(| .*)$\s)+)^# ///$'

def _deps(script: bytes | str) -> dict | None:
    'Get the dependencies from the script. From: https://peps.python.org/pep-0723/'
    name = 'script'
    if isinstance(script, bytes): script = script.decode('utf-8')
    matches = L(re.finditer(pat, script)).filter(lambda m: m.group('type') == name)
    if len(matches) > 1: raise ValueError(f'Multiple {name} blocks found')
    elif len(matches) == 1:
        content = ''.join(line[2:] if line.startswith('# ') else line[1:]
                          for line in matches[0].group('content').splitlines(keepends=True))
        return '\n'.join(tomllib.loads(content)['dependencies'])
    else: return None

In [ ]:
#| hide
# Lets test on some demo apps:
test_eq(_deps(Path("../examples/inline_dependencies/main.py").read_text()),"python-fasthtml")
test_is(_deps(Path("../examples/fasthtml/main.py").read_text()),None)

In [ ]:
#| export
def _validate_app(path):
    "Validates directory `path` is a deployable Plash app"
    if not (path / 'main.py').exists():
        raise PlashError('A Plash app requires a main.py file.')
    deps = _deps((path / 'main.py').read_text(encoding='utf-8'))
    if  deps and (path/"requirements.txt").exists(): 
        raise PlashError('A Plash app should not contain both a requirements.txt file and inline dependencies (see PEP723).')

In [ ]:
#| hide
# All test apps should be valid:
for d in Path("../examples/").iterdir(): test_is(_validate_app(d),None)

In [ ]:
#| hide
# Lets test each failure case:
with TemporaryDirectory() as td:
    td = Path(td)
    test_fail(_validate_app, args=(td,), contains="main.py")
    
    # test failure case of deps in both main.py and requirements.txt
    (td / "requirements.txt").write_text("")
    (td / "main.py").write_text(Path("../examples/inline_dependencies/main.py").read_text())
    test_fail(_validate_app, args=(td,), contains="not contain both")

In [ ]:
#| export
#| hide
def create_tar_archive(path:Path, force_data:bool=False) -> tuple[io.BytesIO, int]:
    "Creates a tar archive of a directory, excluding files based on is_included"
    tarz = io.BytesIO()
    files = L(path if path.is_file() else Path(path).iterdir()).filter(_is_included)
    if not force_data: files = files.filter(lambda f: f.name != 'data')
    with tarfile.open(fileobj=tarz, mode='w:gz') as tar:
        for f in files: tar.add(f, arcname=f.name)
        if deps:=_deps((path / 'main.py').read_bytes()):
            info = tarfile.TarInfo('requirements.txt')
            info.size = len(deps)
            tar.addfile(info, io.BytesIO(deps.encode('utf-8')))
    tarz.seek(0)
    return tarz, len(files)

In [ ]:
#| export
def _gen_app_name():
    adjectives = ['admiring', 'adoring', 'amazing', 'awesome', 'beautiful', 'blissful', 'bold', 'brave', 'busy', 'charming', 'clever', 'compassionate', 'confident', 'cool', 'dazzling', 'determined', 'dreamy', 'eager', 'ecstatic', 'elastic', 'elated', 'elegant', 'epic', 'exciting', 'fervent', 'festive', 'flamboyant', 'focused', 'friendly', 'frosty', 'funny', 'gallant', 'gifted', 'goofy', 'gracious', 'great', 'happy', 'hopeful', 'hungry', 'inspiring', 'intelligent', 'interesting', 'jolly', 'jovial', 'keen', 'kind', 'laughing', 'loving', 'lucid', 'magical', 'modest', 'nice', 'nifty', 'nostalgic', 'objective', 'optimistic', 'peaceful', 'pensive', 'practical', 'priceless', 'quirky', 'quizzical', 'relaxed', 'reverent', 'romantic', 'serene', 'sharp', 'silly', 'sleepy', 'stoic', 'sweet', 'tender', 'trusting', 'upbeat', 'vibrant', 'vigilant', 'vigorous', 'wizardly', 'wonderful', 'youthful', 'zealous', 'zen', 'golden', 'silver', 'crimson', 'azure', 'emerald', 'violet', 'amber', 'coral', 'turquoise', 'lavender', 'minty', 'citrus', 'vanilla', 'woody', 'floral', 'fresh', 'gentle', 'sparkling', 'precise', 'curious']
    nouns = ['tiger', 'eagle', 'river', 'mountain', 'forest', 'ocean', 'star', 'moon', 'wind', 'dragon', 'phoenix', 'wolf', 'bear', 'lion', 'shark', 'falcon', 'raven', 'crystal', 'diamond', 'ruby', 'sapphire', 'pearl', 'wave', 'tide', 'cloud', 'rainbow', 'sunset', 'sunrise', 'galaxy', 'comet', 'meteor', 'planet', 'nebula', 'cosmos', 'universe', 'atom', 'photon', 'quantum', 'matrix', 'cipher', 'code', 'signal', 'pulse', 'beam', 'ray', 'spark', 'frost', 'ice', 'snow', 'mist', 'fog', 'dew', 'rain', 'hail', 'helix', 'prism', 'lens', 'mirror', 'echo', 'heart', 'mind', 'dream', 'vision', 'hope', 'wish', 'magic', 'spell', 'charm', 'rune', 'symbol', 'token', 'key', 'door', 'gate', 'bridge', 'tower', 'castle', 'fortress', 'shield', 'dolphin', 'whale', 'penguin', 'butterfly', 'hummingbird', 'deer', 'rabbit', 'fox', 'otter', 'panda', 'koala', 'zebra', 'giraffe', 'elephant', 'valley', 'canyon', 'meadow', 'prairie', 'island', 'lake', 'pond', 'stream', 'waterfall', 'cliff', 'peak', 'hill', 'grove', 'garden', 'sunlight', 'breeze', 'melody', 'sparkle', 'whirlpool', 'windmill', 'carousel', 'spiral', 'glow']
    verbs = ['runs', 'flies', 'jumps', 'builds', 'creates', 'flows', 'shines', 'grows', 'moves', 'works', 'dances', 'sings', 'plays', 'dreams', 'thinks', 'learns', 'teaches', 'helps', 'heals', 'saves', 'protects', 'guards', 'watches', 'sees', 'hears', 'feels', 'knows', 'understands', 'discovers', 'explores', 'searches', 'finds', 'seeks', 'holds', 'carries', 'lifts', 'pushes', 'pulls', 'makes', 'crafts', 'forges', 'shapes', 'forms', 'molds', 'carves', 'joins', 'connects', 'links', 'binds', 'ties', 'opens', 'closes', 'starts', 'stops', 'begins', 'ends', 'finishes', 'completes', 'wins', 'triumphs', 'succeeds', 'achieves', 'accomplishes', 'reaches', 'arrives', 'departs', 'leaves', 'returns', 'comes', 'goes', 'travels', 'journeys', 'walks', 'sprints', 'races', 'speeds', 'rushes', 'hurries', 'waits', 'pauses', 'rests', 'sleeps', 'wakes', 'rises', 'climbs', 'ascends', 'descends', 'swims', 'dives', 'surfs', 'sails', 'paddles', 'hikes', 'treks', 'wanders', 'roams', 'ventures', 'navigates', 'glides', 'soars', 'floats', 'drifts', 'tosses', 'divides', 'shares', 'secures', 'settles', 'places', 'wonders', 'questions']
    suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=3))
    return f"{random.choice(adjectives)}-{random.choice(nouns)}-{random.choice(verbs)}-{suffix}"

In [ ]:
#| export
@call_parse
def deploy(
    path:Path=Path('.'),    # Path to project
    name:str=None,          # Overrides the .plash file in project root if provided
    force_data:bool=False): # Overwrite data/ directory during deployment
    """
    Deploys app to production. By default, this command erases all files in your app which are not in data/.
    Then uploads all files and folders, except paths starting with `.` and except the local data/ directory.
    If `--force_data` is used, then it erases all files in production. Then it uploads all files and folders,
    including `data/`, except paths starting with `.`.
    """
    print('Initializing deployment...')
    if name == '': print('Error: App name cannot be an empty string'); return
    if not path.is_dir(): print("Error: Path should point to the project directory"); return
    try: _validate_app(path)
    except PlashError as e: print(f"Error: {str(e)}\nInvalid path: {path}"); return
    
    try: 
        if not name: name = _get_app_name(path)
    except FileNotFoundError:
        plash_app = path / '.plash'
        name = _gen_app_name()
        plash_app.write_text(f'export PLASH_APP_NAME={name}')
    
    tarz, _ = create_tar_archive(path, force_data)
    r = _mk_auth_req(_endpoint(rt="/upload"), "post", files={'file': tarz},
                     data={'name': name, 'force_data': force_data})
    if r:
        print('✅ Upload complete! Your app is currently being built.')
        print(f'It will be live at {name if "." in name else _endpoint(sub=name)}')

CLI usage:

In [ ]:
%%bash
plash_deploy --help

usage: plash_deploy [-h] [--path PATH] [--name NAME] [--force_data]

Deploys app to production. By default, this command erases all files in your app which are not in data/. Then uploads
all files and folders, except paths starting with `.` and except the local data/ directory. If `--force_data` is used,
then it erases all files in production. Then it uploads all files and folders, including `data/`, except paths starting
with `.`.

options:
  -h, --help    show this help message and exit
  --path PATH   Path to project (default: .)
  --name NAME   Overrides the .plash file in project root if provided
  --force_data  Overwrite data/ directory during deployment (default: False)


In [ ]:
#| export
@call_parse
def view(
    path:Path=Path('.'), # Path to project directory
    name:str=None,     # Overrides the .plash file in project root if provided
):
    "Open your app in the browser"
    if not name: name = _get_app_name(path)
    url = name if '.' in name else _endpoint(sub=name)
    print(f"Opening browser to view app :\n{url}\n")
    webbrowser.open(url)

CLI usage:

In [ ]:
%%bash
plash_view --help

usage: plash_view [-h] [--path PATH] [--name NAME]

Open your app in the browser

options:
  -h, --help   show this help message and exit
  --path PATH  Path to project directory (default: .)
  --name NAME  Overrides the .plash file in project root if provided


In [ ]:
#| export
@call_parse
def delete(
    path:Path=Path('.'), # Path to project
    name:str=None,     # Overrides the .plash file in project root if provided
    force:bool=False):   # Skip confirmation prompt
    'Delete your deployed app'
    if not name: name = _get_app_name(path)
    if not force:
        confirm = input(f"Are you sure you want to delete app '{name}'? This action cannot be undone. [y/N]: ")
        if confirm.lower() not in ['y', 'yes']:
            print("Deletion cancelled.")
            return
    
    print(f"Deleting app '{name}'...")
    if r := _mk_auth_req(_endpoint(rt=f"/delete?name={name}"), "delete"): return r.text

CLI usage:

In [ ]:
%%bash
plash_delete --help

usage: plash_delete [-h] [--path PATH] [--name NAME] [--force]

Delete your deployed app

options:
  -h, --help   show this help message and exit
  --path PATH  Path to project (default: .)
  --name NAME  Overrides the .plash file in project root if provided
  --force      Skip confirmation prompt (default: False)


In [ ]:
#| export
@call_parse
def start(path:Path=Path('.'), name:str=None):
    "Start your deployed app"
    if not name: name = _get_app_name(path)
    if r := _mk_auth_req(_endpoint(rt=f"/start?name={name}")): return r.text

CLI usage:

In [ ]:
%%bash
plash_start --help

usage: plash_start [-h] [--path PATH] [--name NAME]

Start your deployed app

options:
  -h, --help   show this help message and exit
  --path PATH  (default: .)
  --name NAME


In [ ]:
#| export
@call_parse  
def stop(path:Path=Path('.'), name:str=None):
    "Stop your deployed app" 
    if not name: name = _get_app_name(path)
    if r := _mk_auth_req(_endpoint(rt=f"/stop?name={name}")): return r.text

CLI usage:

In [ ]:
%%bash
plash_stop --help

usage: plash_stop [-h] [--path PATH] [--name NAME]

Stop your deployed app

options:
  -h, --help   show this help message and exit
  --path PATH  (default: .)
  --name NAME


In [ ]:
#| export
log_modes = str_enum('log_modes', 'build', 'app')

In [ ]:
#| export
@call_parse
def logs(
    path:Path=Path('.'),    # Path to project
    name:str=None,          # Overrides the .plash file in project root if provided
    mode:log_modes='build', # Choose between build or app logs
    tail:bool=False):       # Tail the logs
    'Prints the logs for your deployed app'
    if not name: name = _get_app_name(path)
    if tail:
        text = ''
        while True:
            try:
                if r := _mk_auth_req(_endpoint(rt=f"/logs?name={name}&mode={mode}")):
                    print(r.text[len(text):], end='') # Only print updates
                    text = r.text
                    if mode == 'build' and 'Build End Time:' in r.text: break
                    sleep(1)
            except KeyboardInterrupt: return "\nExiting"
    if r := _mk_auth_req(_endpoint(rt=f"/logs?name={name}&mode={mode}")): return r.text

CLI usage:

In [ ]:
%%bash
plash_logs --help

usage: plash_logs [-h] [--path PATH] [--name NAME] [--mode {build,app}] [--tail]

Prints the logs for your deployed app

options:
  -h, --help          show this help message and exit
  --path PATH         Path to project (default: .)
  --name NAME         Overrides the .plash file in project root if provided
  --mode {build,app}  Choose between build or app logs (default: build)
  --tail              Tail the logs (default: False)


In [ ]:
#| hide
import tempfile
temp_dir = tempfile.TemporaryDirectory()
td = Path(temp_dir.name)

In [ ]:
#| export
@patch
def _is_dir_empty(self:Path): return next(self.iterdir(), None) is None

In [ ]:
#| hide
test_eq(td.is_dir_empty(), True)
(td/'.temp').write_text('Hello, world!')
test_eq(d.is_dir_empty(), False)

In [ ]:
#| hide
temp_dir.cleanup()

In [ ]:
#| export
@call_parse
def download(
    path:Path=Path('.'),                 # Path to project
    name:str=None,                       # Overrides the .plash file in project root if provided
    save_path:Path=Path("./download/")): # Save path (optional)
    'Download your deployed app'
    if not name: name = _get_app_name(path)
    save_path.mkdir(exist_ok=True)
    if not save_path._is_dir_empty(): return print(f'ERROR: Save path ({save_path}) is not empty.')
    if r := _mk_auth_req(_endpoint(rt=f'/download?name={name}')):
        with tarfile.open(fileobj=io.BytesIO(r.content), mode="r:gz") as tar: tar.extractall(path=save_path)
        print(f"Downloaded your app to: {save_path}")        

CLI usage:

In [ ]:
%%bash
plash_download --help

usage: plash_download [-h] [--path PATH] [--name NAME] [--save_path SAVE_PATH]

Download your deployed app

options:
  -h, --help             show this help message and exit
  --path PATH            Path to project (default: .)
  --name NAME            Overrides the .plash file in project root if provided
  --save_path SAVE_PATH  Save path (optional) (default: download)


In [ ]:
#|export
@call_parse
def apps(verbose:bool=False):
    "List your deployed apps (verbose shows status table: 1=running, 0=stopped)"
    r = _mk_auth_req(_endpoint(rt="/user_apps"))
    if r:
        apps = r.json()
        if not apps: return "You don't have any deployed Plash apps."
        if verbose: [print(f"{a['running']} {a['name']}") for a in apps]
        else: [print(a['name']) for a in apps]

CLI usage:

In [ ]:
%%bash
plash_apps --help

usage: plash_apps [-h] [--verbose]

List your deployed apps (verbose shows status table: 1=running, 0=stopped)

options:
  -h, --help  show this help message and exit
  --verbose   (default: False)


## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()